In [1]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
def build_discriminator():
    model = Sequential()
    model.add(Dense(41, input_dim=41, activation='relu'))  # discriminator takes 41 values from our dataset
    model.add(Dense(30, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # outputs 0 to 1, 1 being read and 0 being fake

    # model.summary()

    attack = Input(shape=(41,))
    validity = model(attack)

    return Model(attack, validity)

In [3]:
def build_generator(hidden1, hidden2, hidden3):
    model = Sequential()
    model.add(Dense(hidden1, input_dim=41))  # arbitrarily selected 100 for our input noise vector?
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(hidden2))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(hidden3))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(41, activation='relu'))  # outputs a generated vector of the same size as our data (41)

    # model.summary()

    noise = Input(shape=(41,))
    attack = model(noise)
    return Model(noise, attack)

In [4]:
def trainGAN(gen_hidden1, gen_hidden2, gen_hidden3):
    batch_size = 256
    epochs = 7000
    optimizer = Adam(0.0002, 0.5)
    
    dataframe = pd.read_csv('portsweep.csv').sample(500) # sample 100 data points randomly from the csv
    
    # apply "le.fit_transform" to every column (usually only works on 1 column)
    le = LabelEncoder()
    dataframe_encoded = dataframe.apply(le.fit_transform)
    dataset = dataframe_encoded.values
    
    #to visually judge results
    print("Real portsweep attacks:")
    print(dataset[:2])
    
    # Set X as our input data and Y as our label
    X_train = dataset[:, 0:41].astype(float)
    Y_train = dataset[:, 41]
    
    # labels for data. 1 for valid attacks, 0 for fake (generated) attacks
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    # build the discriminator portion
    discriminator = build_discriminator();
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # build the generator portion
    generator = build_generator(gen_hidden1, gen_hidden2, gen_hidden3)
    
    #input and output of our combined model
    z = Input(shape=(41,))
    attack = generator(z)
    validity = discriminator(attack)
    
    # build combined model from generator and discriminator
    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    #break condition for training (when diverging)
    loss_increase_count = 0;
    prev_g_loss = 0;
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        # selecting batch_size random attacks from our training data
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        attacks = X_train[idx]
        
        # generate a matrix of noise vectors
        noise = np.random.normal(0, 1, (batch_size, 41))
        
        # create an array of generated attacks
        gen_attacks = generator.predict(noise)
        
        # loss functions, based on what metrics we specify at model compile time
        d_loss_real = discriminator.train_on_batch(attacks, valid)
        d_loss_fake = discriminator.train_on_batch(gen_attacks, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # generator loss function
        g_loss = combined.train_on_batch(noise, valid)
        
        if epoch % 100 == 0:
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [Loss change: %.3f, Loss increases: %.0f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss, g_loss - prev_g_loss, loss_increase_count))
        
        # if our generator loss icreased this iteration, increment the counter by 1
        if (g_loss - prev_g_loss) > 0:
            loss_increase_count = loss_increase_count + 1
        else: 
            loss_increase_count = 0  # otherwise, reset it to 0, we are still training effectively
            
        prev_g_loss = g_loss
            
        if loss_increase_count > 5:
            print('Stoping on iteration: ', epoch)
            break
            
        if epoch % 20 == 0:
            f = open("GANresultsportsweep.txt", "a")
            np.savetxt("GANresultsportsweep.txt", gen_attacks, fmt="%.0f")
            f.close()
            
    # peek at our results
    results = np.loadtxt("GANresultsportsweep.txt")
    print("Generated portsweep attacks: ")
    print(results[:2])
        
        


In [ ]:
# Initialize Random Number Generator
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load dataset

dataframe = pd.read_csv("normalAndPortsweep.csv")#, header=True) 

# samples 10000 random data points from 500k
dataframe = dataframe.sample(n=10000)
# LabelEncoder, turns all our categorical data into integers
le = LabelEncoder()

# apply "le.fit_transform" to every column (usually only works on 1 column)
dataframe_encoded = dataframe.apply(le.fit_transform)
attack_labels = le.classes_
indices_of_portsweep = np.where(attack_labels == 'portsweep.')
portsweep_index = indices_of_portsweep[0]
dataset = dataframe_encoded.values

print(attack_labels)
print(portsweep_index)

#Set X as our input data and Y as our label
X = dataset[:,0:41].astype(float)
Y = dataset[:,41]


['normal.' 'portsweep.']
[1]


In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)
#print(len(dummy_y[0]))
num_of_classes = len(dummy_y[0])  # the length of dummy y is the number of classes we have in our small sample
# since we are randomly sampling from a large dataset, we might not get 1 of every class in our sample
# we need to set output layer to be equal to the length of our dummy_y vectors
print(num_of_classes)


2


In [ ]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    
    inputs = 41
    hidden_layer1 = 10
    hidden_layer2 = 5
    hidden_layer3 = 0
    outputs = num_of_classes  #needs to be this variable in case we forget to sample. Could end up having 10 classes or 12, etc
    
    model.add(Dense(hidden_layer1, input_dim=inputs, activation='relu'))
    if hidden_layer2 != 0:
        model.add(Dense(hidden_layer2, activation='relu'))
    if hidden_layer3 != 0:
        model.add(Dense(hidden_layer3, activation='relu'))
    model.add(Dense(outputs, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #optimizer=adam
    return model

In [ ]:
#for i in range(0,10):
estimator = KerasClassifier(build_fn=baseline_model, epochs=32, batch_size=200, verbose=2)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
y_pred = cross_val_predict(estimator, X, dummy_y, cv=kfold)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

trained_classifier = estimator.fit(X, Y)
print(type(estimator))

cm = confusion_matrix(Y, y_pred)
print(cm)
print("total: " + str(cm.sum()))
print("accuracy: " + str(np.trace(cm) / cm.sum()))
print("Matthews correlation coefficient: " + str(matthews_corrcoef(Y, y_pred)))



print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

f = open("discriminatorResults.txt", "a+")
f.write("TP: %d, FP: %d, FN: %d, TN: %d\n" % (cm[0][0], cm[0][1], cm[1][0], cm[1][1]))
f.close()


Epoch 1/32
 - 0s - loss: 3.6816 - acc: 0.7439
Epoch 2/32
 - 0s - loss: 0.6404 - acc: 0.9487
Epoch 3/32
 - 0s - loss: 0.3282 - acc: 0.9700
Epoch 4/32
 - 0s - loss: 0.2471 - acc: 0.9774
Epoch 5/32
 - 0s - loss: 0.2162 - acc: 0.9800
Epoch 6/32
 - 0s - loss: 0.2028 - acc: 0.9813
Epoch 7/32
 - 0s - loss: 0.1925 - acc: 0.9834
Epoch 8/32
 - 0s - loss: 0.1847 - acc: 0.9851
Epoch 9/32
 - 0s - loss: 0.1773 - acc: 0.9871
Epoch 10/32
 - 0s - loss: 0.1663 - acc: 0.9878
Epoch 11/32
 - 0s - loss: 0.1346 - acc: 0.9891
Epoch 12/32
 - 0s - loss: 0.1105 - acc: 0.9917
Epoch 13/32
 - 0s - loss: 0.1010 - acc: 0.9923
Epoch 14/32
 - 0s - loss: 0.0846 - acc: 0.9928
Epoch 15/32
 - 0s - loss: 0.0613 - acc: 0.9937
Epoch 16/32
 - 0s - loss: 0.0373 - acc: 0.9953
Epoch 17/32
 - 0s - loss: 0.0264 - acc: 0.9968
Epoch 18/32
 - 0s - loss: 0.0221 - acc: 0.9971
Epoch 19/32
 - 0s - loss: 0.0201 - acc: 0.9978
Epoch 20/32
 - 0s - loss: 0.0191 - acc: 0.9981
Epoch 21/32
 - 0s - loss: 0.0186 - acc: 0.9982
Epoch 22/32
 - 0s - lo

Epoch 17/32
 - 0s - loss: 7.6598 - acc: 0.5247
Epoch 18/32
 - 0s - loss: 7.6598 - acc: 0.5248
Epoch 19/32
 - 0s - loss: 7.6598 - acc: 0.5247
Epoch 20/32
 - 0s - loss: 7.6598 - acc: 0.5247
Epoch 21/32
 - 0s - loss: 7.6598 - acc: 0.5248
Epoch 22/32
 - 0s - loss: 7.6598 - acc: 0.5247
Epoch 23/32
 - 0s - loss: 7.6599 - acc: 0.5248
Epoch 24/32
 - 0s - loss: 7.6599 - acc: 0.5247
Epoch 25/32
 - 0s - loss: 7.6598 - acc: 0.5247
Epoch 26/32
 - 0s - loss: 7.6598 - acc: 0.5248
Epoch 27/32
 - 0s - loss: 7.6599 - acc: 0.5247
Epoch 28/32
 - 0s - loss: 7.6598 - acc: 0.5247
Epoch 29/32
 - 0s - loss: 7.6598 - acc: 0.5248
Epoch 30/32
 - 0s - loss: 7.6598 - acc: 0.5248
Epoch 31/32
 - 0s - loss: 7.6598 - acc: 0.5248
Epoch 32/32
 - 0s - loss: 7.6598 - acc: 0.5247
Epoch 1/32
 - 0s - loss: 4.6954 - acc: 0.6580
Epoch 2/32
 - 0s - loss: 0.1004 - acc: 0.9841
Epoch 3/32
 - 0s - loss: 0.0337 - acc: 0.9957
Epoch 4/32
 - 0s - loss: 0.0285 - acc: 0.9973
Epoch 5/32
 - 0s - loss: 0.0267 - acc: 0.9976
Epoch 6/32
 - 0s -

Epoch 1/32
 - 0s - loss: 7.5778 - acc: 0.5292
Epoch 2/32
 - 0s - loss: 1.7930 - acc: 0.8459
Epoch 3/32
 - 0s - loss: 0.1810 - acc: 0.9729
Epoch 4/32
 - 0s - loss: 0.0262 - acc: 0.9949
Epoch 5/32
 - 0s - loss: 0.0199 - acc: 0.9974
Epoch 6/32
 - 0s - loss: 0.0188 - acc: 0.9976
Epoch 7/32
 - 0s - loss: 0.0177 - acc: 0.9977
Epoch 8/32
 - 0s - loss: 0.0175 - acc: 0.9976
Epoch 9/32
 - 0s - loss: 0.0167 - acc: 0.9977
Epoch 10/32
 - 0s - loss: 0.0159 - acc: 0.9978
Epoch 11/32
 - 0s - loss: 0.0151 - acc: 0.9981
Epoch 12/32
 - 0s - loss: 0.0137 - acc: 0.9980
Epoch 13/32
 - 0s - loss: 0.0133 - acc: 0.9983
Epoch 14/32
 - 0s - loss: 0.0124 - acc: 0.9979
Epoch 15/32
 - 0s - loss: 0.0101 - acc: 0.9981
Epoch 16/32
 - 0s - loss: 0.0097 - acc: 0.9986
Epoch 17/32
 - 0s - loss: 0.0089 - acc: 0.9987
Epoch 18/32
 - 0s - loss: 0.0076 - acc: 0.9992
Epoch 19/32
 - 0s - loss: 0.0068 - acc: 0.9992
Epoch 20/32
 - 0s - loss: 0.0060 - acc: 0.9993
Epoch 21/32
 - 0s - loss: 0.0058 - acc: 0.9993
Epoch 22/32
 - 0s - lo

Epoch 17/32
 - 0s - loss: 0.0138 - acc: 0.9989
Epoch 18/32
 - 0s - loss: 0.0136 - acc: 0.9989
Epoch 19/32
 - 0s - loss: 0.0134 - acc: 0.9989
Epoch 20/32
 - 0s - loss: 0.0133 - acc: 0.9989
Epoch 21/32
 - 0s - loss: 0.0131 - acc: 0.9989
Epoch 22/32
 - 0s - loss: 0.0130 - acc: 0.9989
Epoch 23/32
 - 0s - loss: 0.0129 - acc: 0.9989
Epoch 24/32
 - 0s - loss: 0.0128 - acc: 0.9989
Epoch 25/32
 - 0s - loss: 0.0127 - acc: 0.9989
Epoch 26/32
 - 0s - loss: 0.0126 - acc: 0.9989
Epoch 27/32
 - 0s - loss: 0.0125 - acc: 0.9988
Epoch 28/32
 - 0s - loss: 0.0124 - acc: 0.9988
Epoch 29/32
 - 0s - loss: 0.0124 - acc: 0.9988
Epoch 30/32
 - 0s - loss: 0.0123 - acc: 0.9989
Epoch 31/32
 - 0s - loss: 0.0122 - acc: 0.9989
Epoch 32/32
 - 0s - loss: 0.0121 - acc: 0.9989
Epoch 1/32
 - 0s - loss: 1.7672 - acc: 0.8672
Epoch 2/32
 - 0s - loss: 0.3494 - acc: 0.9619
Epoch 3/32
 - 0s - loss: 0.0645 - acc: 0.9872
Epoch 4/32
 - 0s - loss: 0.0360 - acc: 0.9918
Epoch 5/32
 - 0s - loss: 0.0285 - acc: 0.9930
Epoch 6/32
 - 0s -

In [ ]:
"""
f = open("GeneratorHypersAbove50percentAccuracy.txt", "w")
f.write("""""" Hidden layer counts for Generator model that resulted in over 50% generated attacks labeled correctly:
    ------------------------------------------------------------------------------------------------
    """""")
f.close()
"""

while(1):
    # generate random numbers for the hidden layer sizes of our generator
    gen_hidden1 =  np.random.randint(1, 101)
    gen_hidden2 =  np.random.randint(1, 101)
    gen_hidden3 =  np.random.randint(1, 101)
    
    i = 0
    
    
    # train 5 times on each setup, in case we get unlucky initalization on an otherwise good setup
    while i < 100:
        # create a unique filename in case we want to store the results (good accuracy)
        result_filename = "GANresultsportsweep%.0f%.0f%.0fiter%.0ftry2.txt" % (gen_hidden1, gen_hidden2, gen_hidden3, i)

        trainGAN(gen_hidden1, gen_hidden2, gen_hidden3)
        
        # load generate attacks from file
        results = np.loadtxt("GANresultsportsweep.txt")

        # predict attack lables (as encoded integers)
        y_pred = estimator.predict(results)
        print(y_pred)

        # create appropriate labels for our generated portsweep attacks
        portsweep_labels = np.full((len(results),), portsweep_index[0])

        # convert integer labels back to string, get all unique strings and their count
        predicted_as_label = attack_labels[y_pred]
        unique_labels = np.unique(predicted_as_label)

        for label in unique_labels:
            print("Attack type: %s     number predicted:  %.0f" % (label, len(np.where(predicted_as_label == label)[0])))
    
        print()
        # create a confusion matrix of the results
        cm = confusion_matrix(portsweep_labels, y_pred)
        
        accuracy = np.trace(cm) / cm.sum()
        print(cm)
        print("total: " + str(cm.sum()))
        print("accuracy: " + str(accuracy))
        
        if accuracy > .50:
            f = open("GeneratorHypersAbove50percentAccuracyportsweep.txt", "a")
            f.write("""
            
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: %.3f
Generator hidden layer 1 size: %.0f
Generator hidden layer 2 size: %.0f
Generator hidden layer 3 size: %.0f
Iteration %.0f
Result file name: %s
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~""" % (accuracy, gen_hidden1, gen_hidden2, gen_hidden3, i, result_filename))
            f.close()
            
            f = open(result_filename, "w")
            f.close()
            np.savetxt(result_filename, results, fmt="%.0f")
        
        i = i + 1
            


Real portsweep attacks:
[[ 0  1 15  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1
   0  0  1  1  1  0  0 27  0  0 51 68  0  0  0 67  3  0]
 [ 0  1 15  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1
   0  0  1  1  1  0  0 27  0  0 63 68  0  0  0 67  3  0]]
0 [D loss: 0.377522, acc.: 53.71%] [G loss: 0.597408] [Loss change: 0.597, Loss increases: 0]
100 [D loss: 0.723762, acc.: 50.39%] [G loss: 0.336984] [Loss change: 0.007, Loss increases: 0]
200 [D loss: 0.767525, acc.: 50.20%] [G loss: 0.295951] [Loss change: -0.008, Loss increases: 1]
300 [D loss: 0.744408, acc.: 50.20%] [G loss: 0.298609] [Loss change: 0.004, Loss increases: 1]
400 [D loss: 0.776376, acc.: 50.20%] [G loss: 0.291795] [Loss change: -0.015, Loss increases: 3]
500 [D loss: 0.776703, acc.: 50.20%] [G loss: 0.300163] [Loss change: 0.008, Loss increases: 0]
600 [D loss: 0.765177, acc.: 50.59%] [G loss: 0.301738] [Loss change: -0.001, Loss increases: 0]
700 [D loss: 0.749756, acc.: 49.80%] [G 

1600 [D loss: 1.222955, acc.: 50.00%] [G loss: 0.100376] [Loss change: -0.002, Loss increases: 1]
1700 [D loss: 1.207489, acc.: 49.80%] [G loss: 0.104541] [Loss change: 0.003, Loss increases: 2]
1800 [D loss: 1.214014, acc.: 49.80%] [G loss: 0.101979] [Loss change: -0.005, Loss increases: 1]
1900 [D loss: 1.190517, acc.: 50.00%] [G loss: 0.106756] [Loss change: -0.001, Loss increases: 2]
2000 [D loss: 1.163801, acc.: 49.80%] [G loss: 0.112997] [Loss change: 0.000, Loss increases: 2]
2100 [D loss: 1.147577, acc.: 49.80%] [G loss: 0.118511] [Loss change: -0.000, Loss increases: 3]
2200 [D loss: 1.130774, acc.: 50.00%] [G loss: 0.123349] [Loss change: 0.003, Loss increases: 0]
2300 [D loss: 1.099700, acc.: 50.00%] [G loss: 0.126107] [Loss change: -0.004, Loss increases: 3]
Stoping on iteration:  2361
Generated portsweep attacks: 
[[ 0.  0.  8.  3.  0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  9.  1.  0.  0. 10.  0.  0.  0. 30.
   0.  0.  0.

0 [D loss: 0.404535, acc.: 50.00%] [G loss: 0.552051] [Loss change: 0.552, Loss increases: 0]
100 [D loss: 0.709182, acc.: 50.00%] [G loss: 0.324112] [Loss change: -0.002, Loss increases: 1]
200 [D loss: 0.822215, acc.: 50.00%] [G loss: 0.271797] [Loss change: -0.001, Loss increases: 0]
300 [D loss: 0.837257, acc.: 50.00%] [G loss: 0.255789] [Loss change: 0.009, Loss increases: 0]
400 [D loss: 0.843244, acc.: 50.00%] [G loss: 0.259192] [Loss change: 0.012, Loss increases: 0]
500 [D loss: 0.810628, acc.: 50.00%] [G loss: 0.256694] [Loss change: 0.000, Loss increases: 1]
600 [D loss: 0.812213, acc.: 50.20%] [G loss: 0.263373] [Loss change: 0.009, Loss increases: 1]
700 [D loss: 0.810039, acc.: 50.00%] [G loss: 0.252917] [Loss change: -0.002, Loss increases: 0]
800 [D loss: 0.812270, acc.: 50.00%] [G loss: 0.249652] [Loss change: 0.001, Loss increases: 0]
900 [D loss: 0.780004, acc.: 50.00%] [G loss: 0.257450] [Loss change: -0.001, Loss increases: 1]
1000 [D loss: 0.790392, acc.: 50.00%] 

900 [D loss: 0.759210, acc.: 50.00%] [G loss: 0.282389] [Loss change: 0.010, Loss increases: 0]
1000 [D loss: 0.750661, acc.: 50.00%] [G loss: 0.277937] [Loss change: -0.003, Loss increases: 1]
1100 [D loss: 0.750734, acc.: 50.00%] [G loss: 0.285161] [Loss change: -0.002, Loss increases: 0]
1200 [D loss: 0.735971, acc.: 50.00%] [G loss: 0.290284] [Loss change: 0.004, Loss increases: 0]
1300 [D loss: 0.728395, acc.: 50.00%] [G loss: 0.294728] [Loss change: -0.001, Loss increases: 1]
1400 [D loss: 0.731344, acc.: 50.00%] [G loss: 0.299592] [Loss change: 0.006, Loss increases: 1]
1500 [D loss: 0.725660, acc.: 50.00%] [G loss: 0.297770] [Loss change: -0.004, Loss increases: 1]
Stoping on iteration:  1562
Generated portsweep attacks: 
[[ 5.  0. 31.  8.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   1.  0.  0.  0.  0.  0.  0.  5.  0.  0.  0.  0.  5.  0. 17. 20.  0.  0.
   2.  0.  8.  0. 31.]
 [ 0.  2.  0.  0.  0.  0.  0.  0.  5.  0.  0.  0.  0.  0.  0.  9.  0.  1.
   0.  0.  0.  

0 [D loss: 3.532480, acc.: 34.57%] [G loss: 0.672143] [Loss change: 0.672, Loss increases: 0]
100 [D loss: 0.931375, acc.: 50.00%] [G loss: 0.216498] [Loss change: -0.005, Loss increases: 2]
200 [D loss: 1.340299, acc.: 50.00%] [G loss: 0.129432] [Loss change: 0.007, Loss increases: 0]
300 [D loss: 1.584082, acc.: 50.00%] [G loss: 0.077889] [Loss change: -0.007, Loss increases: 0]
400 [D loss: 1.740018, acc.: 50.00%] [G loss: 0.055466] [Loss change: 0.003, Loss increases: 3]
500 [D loss: 1.767526, acc.: 50.00%] [G loss: 0.046876] [Loss change: 0.006, Loss increases: 0]
600 [D loss: 1.711745, acc.: 50.00%] [G loss: 0.044995] [Loss change: -0.005, Loss increases: 1]
700 [D loss: 1.640867, acc.: 49.61%] [G loss: 0.047046] [Loss change: 0.000, Loss increases: 1]
800 [D loss: 1.647797, acc.: 50.00%] [G loss: 0.047194] [Loss change: -0.009, Loss increases: 1]
900 [D loss: 1.595976, acc.: 49.80%] [G loss: 0.055975] [Loss change: 0.002, Loss increases: 0]
1000 [D loss: 1.603474, acc.: 50.00%] 

...

0 [D loss: 0.373040, acc.: 54.10%] [G loss: 0.498034] [Loss change: 0.498, Loss increases: 0]
100 [D loss: 0.778247, acc.: 50.00%] [G loss: 0.291913] [Loss change: 0.004, Loss increases: 0]
200 [D loss: 0.812394, acc.: 50.00%] [G loss: 0.261735] [Loss change: 0.002, Loss increases: 1]
300 [D loss: 0.840271, acc.: 50.00%] [G loss: 0.253320] [Loss change: -0.003, Loss increases: 0]
400 [D loss: 0.857422, acc.: 50.00%] [G loss: 0.246230] [Loss change: -0.003, Loss increases: 0]
500 [D loss: 0.827745, acc.: 50.00%] [G loss: 0.254621] [Loss change: 0.008, Loss increases: 0]
600 [D loss: 0.829538, acc.: 50.00%] [G loss: 0.257774] [Loss change: 0.011, Loss increases: 0]
700 [D loss: 0.809569, acc.: 50.00%] [G loss: 0.264718] [Loss change: 0.001, Loss increases: 2]
800 [D loss: 0.800086, acc.: 50.00%] [G loss: 0.274438] [Loss change: 0.005, Loss increases: 0]
900 [D loss: 0.774382, acc.: 50.20%] [G loss: 0.282492] [Loss change: 0.007, Loss increases: 0]
1000 [D loss: 0.777088, acc.: 50.00%] [G

1500 [D loss: 1.572253, acc.: 50.00%] [G loss: 0.052915] [Loss change: -0.004, Loss increases: 0]
1600 [D loss: 1.541318, acc.: 50.00%] [G loss: 0.064647] [Loss change: 0.004, Loss increases: 1]
1700 [D loss: 1.514598, acc.: 50.00%] [G loss: 0.062022] [Loss change: 0.001, Loss increases: 0]
1800 [D loss: 1.469286, acc.: 50.00%] [G loss: 0.070458] [Loss change: 0.002, Loss increases: 2]
1900 [D loss: 1.510926, acc.: 50.00%] [G loss: 0.064166] [Loss change: -0.008, Loss increases: 1]
2000 [D loss: 1.474168, acc.: 50.00%] [G loss: 0.071115] [Loss change: -0.004, Loss increases: 0]
2100 [D loss: 1.468830, acc.: 50.00%] [G loss: 0.073537] [Loss change: -0.003, Loss increases: 0]
2200 [D loss: 1.438397, acc.: 50.00%] [G loss: 0.082253] [Loss change: 0.003, Loss increases: 0]
2300 [D loss: 1.446498, acc.: 50.00%] [G loss: 0.082671] [Loss change: 0.000, Loss increases: 0]
2400 [D loss: 1.413859, acc.: 50.00%] [G loss: 0.091669] [Loss change: 0.001, Loss increases: 1]
2500 [D loss: 1.402514, ac

1600 [D loss: 1.052342, acc.: 50.00%] [G loss: 0.158400] [Loss change: 0.002, Loss increases: 0]
1700 [D loss: 1.027516, acc.: 50.00%] [G loss: 0.184525] [Loss change: 0.009, Loss increases: 1]
1800 [D loss: 0.994829, acc.: 50.00%] [G loss: 0.180840] [Loss change: 0.006, Loss increases: 0]
1900 [D loss: 0.976155, acc.: 50.00%] [G loss: 0.192345] [Loss change: 0.005, Loss increases: 2]
2000 [D loss: 0.960054, acc.: 50.00%] [G loss: 0.189485] [Loss change: -0.008, Loss increases: 1]
2100 [D loss: 0.950201, acc.: 50.00%] [G loss: 0.202605] [Loss change: -0.005, Loss increases: 1]
2200 [D loss: 0.899013, acc.: 50.00%] [G loss: 0.215756] [Loss change: 0.007, Loss increases: 0]
2300 [D loss: 0.878925, acc.: 50.00%] [G loss: 0.229607] [Loss change: 0.004, Loss increases: 1]
2400 [D loss: 0.878389, acc.: 50.00%] [G loss: 0.224448] [Loss change: -0.001, Loss increases: 1]
2500 [D loss: 0.866533, acc.: 50.00%] [G loss: 0.231279] [Loss change: -0.003, Loss increases: 0]
2600 [D loss: 0.852005, ac

0 [D loss: 2.824909, acc.: 49.80%] [G loss: 1.021672] [Loss change: 1.022, Loss increases: 0]
100 [D loss: 0.832599, acc.: 50.20%] [G loss: 0.263747] [Loss change: 0.001, Loss increases: 0]
200 [D loss: 1.053557, acc.: 50.00%] [G loss: 0.165264] [Loss change: 0.012, Loss increases: 0]
300 [D loss: 1.228559, acc.: 50.00%] [G loss: 0.119665] [Loss change: -0.007, Loss increases: 2]
400 [D loss: 1.266235, acc.: 49.61%] [G loss: 0.101013] [Loss change: -0.005, Loss increases: 1]
500 [D loss: 1.260131, acc.: 50.00%] [G loss: 0.114367] [Loss change: 0.010, Loss increases: 0]
600 [D loss: 1.243211, acc.: 49.80%] [G loss: 0.100995] [Loss change: -0.007, Loss increases: 1]
700 [D loss: 1.230880, acc.: 49.41%] [G loss: 0.108425] [Loss change: -0.000, Loss increases: 0]
Stoping on iteration:  774
Generated portsweep attacks: 
[[ 0.  0.  7.  0.  0.  0.  0.  0.  0.  0.  5.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  3.  3.  1.  0.  0.  0.  0.  0.  4.  0.  0. 11.  0.  1. 13.  0.
   0.  1.  0.  0.  0.]
 [

300 [D loss: 0.740635, acc.: 50.00%] [G loss: 0.306314] [Loss change: -0.005, Loss increases: 1]
400 [D loss: 0.758291, acc.: 50.20%] [G loss: 0.299982] [Loss change: 0.003, Loss increases: 1]
